In [1]:
# example of training a gan on mnist
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

In [3]:
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [4]:
# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    # upsample to 14x14
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 28x28
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
    return model

In [5]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [6]:
# load and prepare mnist training images
def load_real_samples():
    # load mnist dataset
    (trainX, _), (_, _) = load_data()
    # expand to 3d, e.g. add channels dimension
    X = expand_dims(trainX, axis=-1)
    # convert from unsigned ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [0,1]
    X = X / 255.0
    return X

In [7]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y

In [8]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [9]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

In [10]:
# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

In [11]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)


In [12]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # create training set for the discriminator
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            # update discriminator model weights
            d_loss, _ = d_model.train_on_batch(X, y)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
        # evaluate the model performance, sometimes
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


4/4 [==============================] - 1s 75ms/step
>1, 1/234, d=0.700, g=0.705
4/4 [==============================] - 0s 60ms/step
>1, 2/234, d=0.690, g=0.719
4/4 [==============================] - 0s 57ms/step
>1, 3/234, d=0.684, g=0.742
4/4 [==============================] - 0s 65ms/step
>1, 4/234, d=0.676, g=0.748
4/4 [==============================] - 0s 59ms/step
>1, 5/234, d=0.668, g=0.777
4/4 [==============================] - 0s 73ms/step
>1, 6/234, d=0.662, g=0.781
4/4 [==============================] - 0s 64ms/step
>1, 7/234, d=0.654, g=0.793
4/4 [==============================] - 0s 64ms/step
>1, 8/234, d=0.651, g=0.803
4/4 [==============================] - 0s 61ms/step
>1, 9/234, d=0.647, g=0.803
4/4 [==============================] - 0s 64ms/step
>1, 10/234, d=0.645, g=0.799
4/4 [==============================] - 0s 57ms/step
>1, 11/234, d=0.645, g=0.789
4/4 [==============================] - 0s 60ms/step
>1, 12/234, d=0.645, g=0.774
4/4 [==============================] 

4/4 [==============================] - 0s 65ms/step
>1, 100/234, d=0.078, g=2.113
4/4 [==============================] - 0s 60ms/step
>1, 101/234, d=0.077, g=2.169
4/4 [==============================] - 0s 59ms/step
>1, 102/234, d=0.080, g=2.211
4/4 [==============================] - 0s 66ms/step
>1, 103/234, d=0.071, g=2.264
4/4 [==============================] - 0s 69ms/step
>1, 104/234, d=0.071, g=2.311
4/4 [==============================] - 0s 63ms/step
>1, 105/234, d=0.071, g=2.345
4/4 [==============================] - 0s 60ms/step
>1, 106/234, d=0.064, g=2.406
4/4 [==============================] - 0s 58ms/step
>1, 107/234, d=0.064, g=2.453
4/4 [==============================] - 0s 64ms/step
>1, 108/234, d=0.060, g=2.483
4/4 [==============================] - 0s 71ms/step
>1, 109/234, d=0.055, g=2.532
4/4 [==============================] - 0s 59ms/step
>1, 110/234, d=0.060, g=2.566
4/4 [==============================] - 0s 65ms/step
>1, 111/234, d=0.052, g=2.610
4/4 [===========

4/4 [==============================] - 0s 64ms/step
>1, 200/234, d=0.134, g=2.278
4/4 [==============================] - 0s 60ms/step
>1, 201/234, d=0.130, g=2.284
4/4 [==============================] - 0s 68ms/step
>1, 202/234, d=0.121, g=2.348
4/4 [==============================] - 0s 50ms/step
>1, 203/234, d=0.120, g=2.388
4/4 [==============================] - 0s 54ms/step
>1, 204/234, d=0.119, g=2.422
4/4 [==============================] - 0s 48ms/step
>1, 205/234, d=0.112, g=2.379
4/4 [==============================] - 0s 61ms/step
>1, 206/234, d=0.111, g=2.418
4/4 [==============================] - 0s 61ms/step
>1, 207/234, d=0.123, g=2.443
4/4 [==============================] - 0s 57ms/step
>1, 208/234, d=0.103, g=2.416
4/4 [==============================] - 0s 63ms/step
>1, 209/234, d=0.103, g=2.430
4/4 [==============================] - 0s 49ms/step
>1, 210/234, d=0.113, g=2.432
4/4 [==============================] - 0s 59ms/step
>1, 211/234, d=0.099, g=2.434
4/4 [===========

4/4 [==============================] - 0s 53ms/step
>2, 67/234, d=0.028, g=3.498
4/4 [==============================] - 0s 53ms/step
>2, 68/234, d=0.028, g=3.516
4/4 [==============================] - 0s 59ms/step
>2, 69/234, d=0.027, g=3.514
4/4 [==============================] - 0s 59ms/step
>2, 70/234, d=0.032, g=3.558
4/4 [==============================] - 0s 53ms/step
>2, 71/234, d=0.025, g=3.552
4/4 [==============================] - 0s 61ms/step
>2, 72/234, d=0.025, g=3.551
4/4 [==============================] - 0s 64ms/step
>2, 73/234, d=0.024, g=3.590
4/4 [==============================] - 0s 64ms/step
>2, 74/234, d=0.028, g=3.582
4/4 [==============================] - 0s 71ms/step
>2, 75/234, d=0.026, g=3.596
4/4 [==============================] - 0s 71ms/step
>2, 76/234, d=0.024, g=3.582
4/4 [==============================] - 0s 60ms/step
>2, 77/234, d=0.025, g=3.637
4/4 [==============================] - 0s 55ms/step
>2, 78/234, d=0.024, g=3.622
4/4 [=======================

4/4 [==============================] - 0s 57ms/step
>2, 167/234, d=0.013, g=4.115
4/4 [==============================] - 0s 58ms/step
>2, 168/234, d=0.017, g=3.742
4/4 [==============================] - 0s 53ms/step
>2, 169/234, d=0.035, g=2.949
4/4 [==============================] - 0s 54ms/step
>2, 170/234, d=0.033, g=3.045
4/4 [==============================] - 0s 56ms/step
>2, 171/234, d=0.032, g=3.060
4/4 [==============================] - 0s 61ms/step
>2, 172/234, d=0.041, g=3.098
4/4 [==============================] - 0s 58ms/step
>2, 173/234, d=0.029, g=3.077
4/4 [==============================] - 0s 57ms/step
>2, 174/234, d=0.032, g=3.152
4/4 [==============================] - 0s 63ms/step
>2, 175/234, d=0.028, g=3.218
4/4 [==============================] - 0s 56ms/step
>2, 176/234, d=0.031, g=3.244
4/4 [==============================] - 0s 60ms/step
>2, 177/234, d=0.030, g=3.327
4/4 [==============================] - 0s 57ms/step
>2, 178/234, d=0.027, g=3.358
4/4 [===========

4/4 [==============================] - 0s 59ms/step
>3, 33/234, d=0.013, g=4.538
4/4 [==============================] - 0s 63ms/step
>3, 34/234, d=0.009, g=4.582
4/4 [==============================] - 0s 60ms/step
>3, 35/234, d=0.009, g=4.534
4/4 [==============================] - 0s 64ms/step
>3, 36/234, d=0.010, g=4.557
4/4 [==============================] - 0s 67ms/step
>3, 37/234, d=0.013, g=4.577
4/4 [==============================] - 0s 60ms/step
>3, 38/234, d=0.008, g=4.592
4/4 [==============================] - 0s 59ms/step
>3, 39/234, d=0.007, g=4.590
4/4 [==============================] - 0s 62ms/step
>3, 40/234, d=0.008, g=4.607
4/4 [==============================] - 0s 57ms/step
>3, 41/234, d=0.009, g=4.580
4/4 [==============================] - 0s 63ms/step
>3, 42/234, d=0.007, g=4.634
4/4 [==============================] - 0s 58ms/step
>3, 43/234, d=0.008, g=4.621
4/4 [==============================] - 0s 66ms/step
>3, 44/234, d=0.009, g=4.657
4/4 [=======================

4/4 [==============================] - 0s 54ms/step
>3, 134/234, d=0.005, g=5.071
4/4 [==============================] - 0s 61ms/step
>3, 135/234, d=0.007, g=5.006
4/4 [==============================] - 0s 57ms/step
>3, 136/234, d=0.004, g=5.082
4/4 [==============================] - 0s 64ms/step
>3, 137/234, d=0.005, g=5.067
4/4 [==============================] - 0s 60ms/step
>3, 138/234, d=0.005, g=5.130
4/4 [==============================] - 0s 57ms/step
>3, 139/234, d=0.008, g=5.128
4/4 [==============================] - 0s 65ms/step
>3, 140/234, d=0.005, g=5.106
4/4 [==============================] - 0s 60ms/step
>3, 141/234, d=0.005, g=5.108
4/4 [==============================] - 0s 61ms/step
>3, 142/234, d=0.004, g=5.115
4/4 [==============================] - 0s 64ms/step
>3, 143/234, d=0.004, g=5.162
4/4 [==============================] - 0s 56ms/step
>3, 144/234, d=0.005, g=5.144
4/4 [==============================] - 0s 56ms/step
>3, 145/234, d=0.005, g=5.161
4/4 [===========

KeyboardInterrupt: 

In [ ]:
# example of loading the generator model and generating images
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot


In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input


In [ ]:
# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, n):
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    pyplot.show()

# load model
model = load_model('generator_model_100.h5')
# generate images
latent_points = generate_latent_points(100, 25)
# generate images
X = model.predict(latent_points)
# plot the result
save_plot(X, 5)